# Calcolo delle extended pareto grid
Lavoriamo sul toro piatto $T=[0,2\pi]\times[0,2\pi]/(x,0)=(x,2\pi),(0,y)=(2\pi,y)$ con funzioni a valori in $\mathbb{R}^2$ polinomiali periodiche, ovvero in seno e costeno, per esempio del tipo $(f_1(x,y),f_2(x,y))=(a\ cos(x)^2+b\ cos(y),c\ sin(x)^3+d\ sin(y)^2)$.

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as scipy
from scipy import optimize
import numdifftools as nd

In [22]:
#------copio dal main------
class Torus:

    #CONSTRUCTOR
    def __init__(self, precision):
        #il più grande sia il valore di precision più punti del toro considero
        self.precision = precision
        eps = 1/precision

        #Questo salva un array di coordinate x e un array di coordinate y di punti del toro
        #creo una griglia di punti: perche'?
        self.point_coords = np.mgrid[0:1:eps, 0:1:eps]

    #GETTERS
    def get_point_coords(self):
        return self.point_coords


In [43]:
#faccio un esempio con la funzione a*np.cos(x)**2+b*np.cos(y),c*np.sin(x)**3-d*np.sin(y)**2
def f_1(x):
    return (2*np.cos(x[0])**2+4*np.cos(x[1]),0)

def f_2(x):
    return 1*np.sin(x[0])**3+3*np.sin(x[1])**2

def f(x,a,b,c,d):
    return (a*np.cos(x[0])**2+b*np.cos(x[1]),c*np.sin(x[0])**3+d*np.sin(x[1])**2)

Adesso l'idea è provare a calcolare i gradienti e dove si annullano usando delle belle funzioncine pronte, ma ho dei problemi perchè non funzionano.
successivamente usando questi risultati voglio vedere come vengono fuori le "pre-EPG" per questa specifica classe di funzioni.
Fatto ciò poi vorrei vedere:

    -come manipolare in modo sensato la nube di punti della pre-EPG in modo da fare una EPG abbastanza fedele: il fato è che usare i metodi di interpolazione potrebbe non avere senso a livello teorico per il risultato che vogliamo ottenere

    -provare a vedere se per la nostra classe di fuzioni viene meglio se facciamo noi una funzione che calcola il gradiente e una che trova gli zeri (secondo me il calcolo del gradiente fatto da noi ha senso perchè la funzione è molto particolare, quello degli zeri no)
    
    -se riusciamo a estendere la classe di funzioni ammissibili

Utilizzo per calcolare gli zeri di funzioni delle funzioni dalla libreria scipy: consultare la [documentazione](https://docs.scipy.org/doc/scipy/reference/optimize.html#id2).

Provo a lavorare su un $\epsilon$-net del toro, perchè le funzioni che volevo usare sopra mi trovano solo una radice.
    
    -costruisco la $\epsilon$-net del toro

    -spero ci siano dei punti vicini ad essere critici

In [54]:
#griglia sul toro

eps=2*np.pi/10
x=[]
for i in np.arange(0,2*np.pi+eps,eps):
    for j in np.arange(0,2*np.pi+eps,eps):
        x.append((i,j))
l=len(x)
# plt.scatter(x,y)

In [1]:
#provo a calcolare i gradienti con la libreria
gr1=nd.Gradient(f_1)
gr2=nd.Gradient(f_2)
#trovo i punti critici, so di per certo che questi sono endpoints of the arcs of the EPG

#faccio il conto degli zeri di f per vedere se punziona
# x=scipy.optimize.fsolve(f_1,(np.pi,0),(1,1))
# print(f_1({x}))
for i in range (0,l+1,1):
    print(gr1(x[i]))

NameError: name 'nd' is not defined

In [ ]:
#calcolo dei punti critici